In [1]:
#using Pkg
#Pkg.add("Distances")
#Pkg.add("Random")
using Distributed
using Distances

# add k workers
k=15
addprocs(k)
nprocs()
@everywhere using Random

# compute cost of tsp tour
@everywhere function cost_tour(d::Array{Float64,2}, o::Array{Int64,1})
    n=length(o)
    result = d[o[n],o[1]]
    for i in 1:(n-1)
        result += d[o[i],o[i+1]]
    end
    return result
end

# compute iterations random permutations, report the best one
@everywhere function random_permutations(n,d,iterations)
    #o=collect(1:n)
    #o=shuffle(o)
    o=randperm(n)
    optcost=cost_tour(d,o)
    optrandom=o
    # println("random permutation iteration 0 tour $o cost: $optcost")
    for j in 1:(iterations-1)
        o=shuffle(o)
        c=cost_tour(d,o)
        if c<optcost
            optcost=c
            optrandom=o
            # println("random permutation iteration $j tour $o cost: $optcost")
        end
    end
    return (optcost, optrandom)
end

function dist_random_permutations(n,d,iterations)
    d_iterations = Int64(iterations/nworkers())
    futures = Array{Future}(undef, nworkers())
    for (i, id) in enumerate(workers())
        futures[i] = @spawnat id random_permutations(n,d,d_iterations)
    end
    p = fetch.(futures)
    optcost = p[1][1]
    optperm = p[1][2]
    for i in 2:length(p)
        if p[i][1]<optcost
            optcost = p[i][1]
            optperm = p[i][2] 
        end
    end
    return (optcost,optperm)
end

dist_random_permutations (generic function with 1 method)

In [2]:
# create n cities
n=100 # cities
x = rand(2,n) # 2d-world

# compute Euclidean distance between cities 
d=pairwise(Euclidean(), x, x)

iterations = Int64(2*3*4*5*6*7*8*9*10)

3628800

In [3]:
nprocs()

16

In [4]:
@time random_permutations(n,d,iterations)

  3.762925 seconds (3.82 M allocations: 3.038 GiB, 1.14% gc time)


(40.70886213638635, [44, 22, 70, 67, 100, 99, 52, 26, 54, 83  …  89, 78, 29, 74, 8, 45, 95, 69, 38, 14])

In [5]:
@time random_permutations(n,d,iterations)

  3.684492 seconds (3.63 M allocations: 3.028 GiB, 1.18% gc time)


(40.46740988482423, [70, 37, 82, 77, 45, 30, 93, 55, 81, 36  …  99, 19, 85, 21, 95, 68, 13, 2, 62, 50])

In [6]:
@time random_permutations(n,d,iterations)

  3.618330 seconds (3.63 M allocations: 3.028 GiB, 1.03% gc time)


(40.780034003664895, [80, 43, 20, 84, 53, 56, 65, 22, 52, 19  …  25, 78, 95, 58, 88, 81, 31, 50, 54, 98])

In [7]:
@time dist_random_permutations(n,d,iterations)

  4.080153 seconds (1.41 M allocations: 71.689 MiB, 0.36% gc time)


(40.54907415852216, [9, 52, 44, 35, 2, 61, 89, 74, 86, 5  …  37, 22, 98, 6, 47, 79, 84, 66, 92, 23])

In [8]:
@time dist_random_permutations(n,d,iterations)

  1.003786 seconds (2.11 k allocations: 90.781 KiB)


(39.406385468679524, [30, 76, 73, 96, 29, 27, 58, 25, 42, 22  …  86, 98, 32, 23, 61, 56, 59, 38, 35, 10])

In [9]:
@time dist_random_permutations(n,d,iterations)

  1.001686 seconds (2.11 k allocations: 90.797 KiB)


(40.53732193711298, [43, 46, 18, 56, 91, 7, 3, 92, 32, 44  …  73, 71, 93, 38, 48, 23, 28, 33, 88, 76])